In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/data_cleaned.csv', low_memory=False)

In [3]:
df.columns

Index(['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth',
       'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language',
       'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province',
       'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode',
       'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders',
       'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors',
       'VehicleIntroDate', 'AlarmImmobiliser', 'TrackingDevice',
       'CapitalOutstanding', 'NewVehicle', 'SumInsured', 'TermFrequency',
       'CalculatedPremiumPerTerm', 'ExcessSelected', 'CoverCategory',
       'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass',
       'StatutoryRiskType', 'TotalPremium', 'TotalClaims', 'log_cubiccapacity',
       'log_kilowatts', 'sqrt_mmcode', 'scaled_log_cubiccapacity',
       'scaled_log_kilowatts', 'scaled_sqrt_mmcode'],
      dtype='object')

In [4]:
missing_data = df.isnull().sum()
print(missing_data[missing_data > 0])

Series([], dtype: int64)


all columns have zero missing values.

In [5]:
df['VehicleType_Model'] = df['VehicleType'].astype(str) + "_" + df['Model'].astype(str)

In [6]:
df['RegYear_VehicleType'] = df['RegistrationYear'].astype(str) + "_" + df['VehicleType'].astype(str)

### Aggregated Features

In [7]:
avg_premium_by_make = df.groupby('make')['TotalPremium'].mean().reset_index()
avg_premium_by_make.columns = ['make', 'AvgPremiumByMake']
df = df.merge(avg_premium_by_make, on='make', how='left')

In [8]:
total_claims_by_make = df.groupby('make')['TotalClaims'].sum().reset_index()
total_claims_by_make.columns = ['make', 'TotalClaimsByMake']
df = df.merge(total_claims_by_make, on='make', how='left')

In [9]:
df['VehicleIntroDate'] = pd.to_datetime(df['VehicleIntroDate'], format='%Y-%m-%d')


ValueError: time data "6/2002" doesn't match format "%Y-%m-%d", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [12]:
df['VehicleIntroDate'] = pd.to_datetime(df['VehicleIntroDate'])
df['VehicleIntroYear'] = df['VehicleIntroDate'].dt.year
df['VehicleIntroMonth'] = df['VehicleIntroDate'].dt.month
df['VehicleIntroDay'] = df['VehicleIntroDate'].dt.day

### Encoding Categorical Data

In [14]:
df = pd.get_dummies(df, columns=['VehicleType', 'make', 'Model'])

In [16]:
df.columns

Index(['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth',
       'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language',
       'AccountType', 'MaritalStatus',
       ...
       'Model_VOLARE W8 28 SEAT B/S B/S', 'Model_X TRAIL 2.0 4X2 XE (R79/R85)',
       'Model_YARIS 1.3 5Dr', 'Model_YARIS T1 3Dr', 'Model_YARIS T1 5Dr',
       'Model_YARIS T3 A/C', 'Model_YARIS T3 SPIRIT 5Dr', 'Model_YARIS T3+',
       'Model_YARIS ZEN3', 'Model_YARIS ZEN3 ACS'],
      dtype='object', length=517)